### 1 - Import test data

In [1]:
import pandas as pd
import pandas_ta as ta

df = pd.read_csv("nasdaq.csv")
# df["Datetime"]=df["Datetime"].str.replace("-05:00","")
df['Datetime']=pd.to_datetime(df['Datetime'], format='ISO8601', utc=True)

df=df[df.High!=df.Low]
df.set_index("Datetime", inplace=True)


In [2]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=12)
df['RSI']=ta.rsi(df.Close, length=10)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=3)
df=df.join(my_bbands)
df

Open          High           Low  \
Datetime                                                              
2024-01-16 14:30:00+00:00  14905.665039  14938.710938  14881.009766   
2024-01-16 14:45:00+00:00  14876.389648  14910.701172  14866.360352   
2024-01-16 15:00:00+00:00  14902.736328  14934.326172  14892.898438   
2024-01-16 15:15:00+00:00  14934.433594  14977.805664  14925.548828   
2024-01-16 15:30:00+00:00  14974.114258  14992.057617  14954.064453   
...                                 ...           ...           ...   
2024-03-11 18:45:00+00:00  16057.845703  16076.823242  16053.979492   
2024-03-11 19:00:00+00:00  16057.684570  16058.118164  16030.704102   
2024-03-11 19:15:00+00:00  16030.795898  16034.460938  16012.697266   
2024-03-11 19:30:00+00:00  16017.302734  16034.933594  16007.973633   
2024-03-11 19:45:00+00:00  16032.431641  16040.884766  16003.586914   

                                  Close     Volume  Dividends  Stock Splits  \
Datetime                                                                      
2024-01-16 14:30:00+00:00  14885.206055  242955046        0.0           0.0   
2024-01-16 14:45:00+00:00  14910.701172  280490855        0.0           0.0   
2024-01-16 15:00:00+00:00  14931.666016  221621025        0.0           0.0   
2024-01-16 15:15:00+00:00  14974.386719  244870000        0.0           0.0   
2024-01-16 15:30:00+00:00  14987.386719  265508000        0.0           0.0   
...                                 ...        ...        ...           ...   
2024-03-11 18:45:00+00:00  16059.698242   94816000        0.0           0.0   
2024-03-11 19:00:00+00:00  16030.704102  107438000        0.0           0.0   
2024-03-11 19:15:00+00:00  16017.939453  128685000        0.0           0.0   
2024-03-11 19:30:00+00:00  16033.071289  160824000        0.0           0.0   
2024-03-11 19:45:00+00:00  16021.193359  311782000        0.0           0.0   

                               EMA_slow      EMA_fast        RSI        ATR  \
Datetime                                                                      
2024-01-16 14:30:00+00:00           NaN           NaN        NaN        NaN   
2024-01-16 14:45:00+00:00           NaN           NaN        NaN        NaN   
2024-01-16 15:00:00+00:00           NaN           NaN        NaN        NaN   
2024-01-16 15:15:00+00:00           NaN           NaN        NaN  47.170590   
2024-01-16 15:30:00+00:00           NaN           NaN        NaN  43.358428   
...                                 ...           ...        ...        ...   
2024-03-11 18:45:00+00:00  16101.475515  16049.304815  51.092717  23.828753   
2024-03-11 19:00:00+00:00  16098.700165  16046.443167  43.394325  25.550549   
2024-03-11 19:15:00+00:00  16095.533079  16042.057980  40.415499  24.288256   
2024-03-11 19:30:00+00:00  16093.083597  16040.675412  45.356249  25.178825   
2024-03-11 19:45:00+00:00  16090.264372  16037.678173  42.297257  29.218500   

                             BBL_15_1.5    BBM_15_1.5    BBU_15_1.5  \
Datetime                                                              
2024-01-16 14:30:00+00:00           NaN           NaN           NaN   
2024-01-16 14:45:00+00:00           NaN           NaN           NaN   
2024-01-16 15:00:00+00:00           NaN           NaN           NaN   
2024-01-16 15:15:00+00:00           NaN           NaN           NaN   
2024-01-16 15:30:00+00:00           NaN           NaN           NaN   
...                                 ...           ...           ...   
2024-03-11 18:45:00+00:00  16021.071082  16044.040495  16067.009907   
2024-03-11 19:00:00+00:00  16024.741283  16045.150977  16065.560670   
2024-03-11 19:15:00+00:00  16022.035729  16044.290430  16066.545130   
2024-03-11 19:30:00+00:00  16021.272409  16042.110091  16062.947774   
2024-03-11 19:45:00+00:00  16018.775612  16041.006966  16063.238321   

                           BBB_15_1.5  BBP_15_1.5  
Datetime                                           
2024-01-16 14:30:00+00:00  

In [3]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-500:-1] 
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 499/499 [00:00<00:00, 4065.65it/s]
/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_59803/961485937.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0


In [4]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):

            return 1
    return 0

df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 499/499 [00:00<00:00, 1912.96it/s]
/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_59803/2955075913.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)


In [5]:
df[df.TotalSignal != 0].head(20)

Datetime          Open          High           Low  \
1   2024-02-12 19:45:00+00:00  15954.995117  15955.016602  15924.980469   
6   2024-02-13 14:30:00+00:00  15604.243164  15667.214844  15591.212891   
32  2024-02-14 14:30:00+00:00  15782.773438  15797.114258  15735.767578   
33  2024-02-14 14:45:00+00:00  15757.331055  15772.385742  15742.649414   
34  2024-02-14 15:00:00+00:00  15775.785156  15784.043945  15758.648438   
35  2024-02-14 15:15:00+00:00  15767.983398  15811.278320  15764.199219   
53  2024-02-14 19:45:00+00:00  15777.642578  15785.403320  15770.233398   
54  2024-02-14 20:00:00+00:00  15785.474609  15798.827148  15780.088867   
85  2024-02-16 14:45:00+00:00  15882.270508  15888.250000  15776.990234   
86  2024-02-16 15:00:00+00:00  15768.183594  15801.506836  15755.662109   
87  2024-02-16 15:15:00+00:00  15794.060547  15817.140625  15781.192383   
105 2024-02-16 19:45:00+00:00  15874.476562  15874.553711  15798.364258   
106 2024-02-16 20:00:00+00:00  15815.395508  15820.114258  15793.748047   
107 2024-02-16 20:15:00+00:00  15803.046875  15818.819336  15797.847656   
108 2024-02-16 20:30:00+00:00  15814.406250  15816.942383  15779.965820   
131 2024-02-20 19:45:00+00:00  15579.941406  15606.277344  15579.941406   
132 2024-02-20 20:00:00+00:00  15603.469727  15619.105469  15588.639648   
133 2024-02-20 20:15:00+00:00  15605.475586  15617.949219  15598.414062   
135 2024-02-20 20:45:00+00:00  15598.508789  15635.104492  15598.508789   
160 2024-02-21 20:30:00+00:00  15483.788086  15556.931641  15483.788086   

            Close     Volume  Dividends  Stock Splits      EMA_slow  \
1    15929.452148  106995000        0.0           0.0  15960.193529   
6    15651.308594  231922623        0.0           0.0  15947.117829   
32   15754.744141  209780119        0.0           0.0  15766.768199   
33   15766.447266  258309263        0.0           0.0  15766.755613   
34   15767.829102  232963702        0.0           0.0  15766.797711   
35   15806.371094  194460000        0.0           0.0  15768.349608   
53   15785.403320  103077000        0.0           0.0  15759.833278   
54   15793.881836  117318000        0.0           0.0  15761.168516   
85   15791.725586  295300424        0.0           0.0  15832.559201   
86   15795.830078  233697951        0.0           0.0  15831.118843   
87   15796.042969  193628000        0.0           0.0  15829.743319   
105  15815.350586  127260000        0.0           0.0  15849.057082   
106  15803.092773  110927000        0.0           0.0  15847.254560   
107  15813.735352  120780000        0.0           0.0  15845.940081   
108  15783.688477  169496000        0.0           0.0  15843.498842   
131  15601.005859  107043000        0.0           0.0  15680.959028   
132  15606.015625  113165000        0.0           0.0  15678.020071   
133  15614.215820  134353000        0.0           0.0  15675.517944   
135  15635.104492  296986000        0.0           0.0  15671.001983   
160  15547.295898  163059000        0.0           0.0  15571.899031   

         EMA_fast        RSI         ATR    BBL_15_1.5    BBM_15_1.5  \
1    15988.775641  27.449467   32.464460  15947.822940  16016.997005   
6    15922.096636  10.811489  135.093314  15823.549635  15962.352214   
32   15654.013820  62.943851   72.923966  15569.243897  15648.160807   
33   15671.311273  64.292822   58.528087  15564.370624  15651.398177   
34   15686.160170  64.462545   47.483894  15560.357090  15657.441146   
35   15704.654158  69.025302   47.348963  15553.616858  15665.263086   
53   15758.819831  61.335656   23.778007  15718.151115  15750.718359   
54   15764.213986  63.276625   22.098098  15718.451112  15754.367839   
85   15868.898396  30.912756   55.560626  15837.261467  15878.780469   
86   15857.657117  32.379439   52.321993  15825.850979  15875.882812   
87   15848.178017  32.462069   46.864076  15812.651875  15870.956576   
105  15865.286528  36.612586   39.110569  15837.445768  15871.416341   
106  15855.718258

In [6]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_59803/2345118794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'],
                           line=dict(color='green', width=1),
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'],
                           line=dict(color='green', width=1),
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'],
                           line=dict(color='black', width=1),
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'],
                           line=dict(color='blue', width=1),
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [8]:
def SIGNAL():
    return df.TotalSignal

In [11]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 1
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16

    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

# df.index = pd.to_datetime(df.index)
print(df)
bt = Backtest(df, MyStrat, cash=1000000, margin=1/30)

                     Datetime          Open          High           Low  \
0   2024-02-12 19:30:00+00:00  15950.669922  15964.536133  15941.274414   
1   2024-02-12 19:45:00+00:00  15954.995117  15955.016602  15924.980469   
2   2024-02-12 20:00:00+00:00  15929.451172  15957.788086  15928.373047   
3   2024-02-12 20:15:00+00:00  15955.330078  15965.315430  15950.684570   
4   2024-02-12 20:30:00+00:00  15957.508789  15962.569336  15948.444336   
..                        ...           ...           ...           ...   
494 2024-03-11 18:30:00+00:00  16054.590820  16057.852539  16034.976562   
495 2024-03-11 18:45:00+00:00  16057.845703  16076.823242  16053.979492   
496 2024-03-11 19:00:00+00:00  16057.684570  16058.118164  16030.704102   
497 2024-03-11 19:15:00+00:00  16030.795898  16034.460938  16012.697266   
498 2024-03-11 19:30:00+00:00  16017.302734  16034.933594  16007.973633   

            Close     Volume  Dividends  Stock Splits      EMA_slow  \
0    15955.817383  104696000

/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_59803/1976978894.py:38: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [12]:
bt.run()

Start                                     0.0
End                                     498.0
Duration                                498.0
Exposure Time [%]                   46.492986
Equity Final [$]                999519.257464
Equity Peak [$]                1000028.330078
Return [%]                          -0.048074
Buy & Hold Return [%]                0.484174
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.060838
Avg. Drawdown [%]                   -0.060838
Max. Drawdown Duration                  495.0
Avg. Drawdown Duration                  495.0
# Trades                                 21.0
Win Rate [%]                        38.095238
Best Trade [%]                       1.432744
Worst Trade [%]                     -2.299069
Avg. Trade [%]                    

In [14]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1966', ...)